In [6]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [7]:
df = pd.read_csv("Data.csv")
df.dtypes

Direksi Perempuan      int64
Direksi Laki-Laki      int64
Proporsi Perempuan    object
Harga penutupan        int64
Total Shares           int64
Total Utang            int64
Total Asset            int64
dtype: object

In [8]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
# Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [9]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,155.000000,155.000000
mean,0.191673,1.285914
std,0.153234,1.987625
min,0.000000,0.159993
25%,0.038462,0.929432
50%,0.200000,0.973140
75%,0.285714,1.124461
max,0.750000,18.327231


In [10]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [11]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1195
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.730
Time:                        00:10:25   Log-Likelihood:                -325.85
No. Observations:                 155   AIC:                             655.7
Df Residuals:                     153   BIC:                             661.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3554      0.257      5.276      0.000       0.848       1.863
X             -0.3624      1.048     -0.346      0.730      -2.433       1.709
==============================================================================
Omnibus:                      266.887   Durbin-Watson:                   0.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30367.419
Skew:                           8.127   Prob(JB):                         0.00
Kurtosis:                      69.617   Cond. No.                         6.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [13]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.070
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.303
Time:                        00:10:25   Log-Likelihood:                -312.89
No. Observations:                 155   AIC:                             629.8
Df Residuals:                     153   BIC:                             635.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6795      0.236      2.875      0.005       0.213       1.146
X              0.9972      0.964      1.034      0.303      -0.908       2.902
==============================================================================
Omnibus:                      272.210   Durbin-Watson:                   0.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33434.041
Skew:                           8.411   Prob(JB):                         0.00
Kurtosis:                      72.957   Cond. No.                         6.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [15]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [16]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [17]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [18]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [19]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [20]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [21]:
data_sample.describe()

,X,Y
count,131.000000,131.000000
mean,0.204778,1.007477
std,0.144268,0.123375
min,0.000000,0.688744
25%,0.105556,0.931717
50%,0.200000,0.970444
75%,0.292857,1.058174
max,0.545455,1.385464


In [22]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.661
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.200
Time:                        00:10:26   Log-Likelihood:                 89.580
No. Observations:                 131   AIC:                            -175.2
Df Residuals:                     129   BIC:                            -169.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9877      0.019     52.772      0.000       0.951       1.025
X              0.0964      0.075      1.289      0.200      -0.052       0.244
==============================================================================
Omnibus:                       20.861   Durbin-Watson:                   0.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.200
Skew:                           1.030   Prob(JB):                     3.37e-06
Kurtosis:                       3.608   Cond. No.                         7.26
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
bX = data_sample["X"] * result.params["X"]

In [24]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [25]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.2071
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.650
Time:                        00:10:26   Log-Likelihood:                 153.10
No. Observations:                 131   AIC:                            -302.2
Df Residuals:                     129   BIC:                            -296.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0919      0.012      7.972      0.000       0.069       0.115
X              0.0210      0.046      0.455      0.650      -0.070       0.112
==============================================================================
Omnibus:                       41.843   Durbin-Watson:                   1.244
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               74.316
Skew:                           1.482   Prob(JB):                     7.29e-17
Kurtosis:                       5.197   Cond. No.                         7.26
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""